In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.distributed as dist

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Utiliy Functions.
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [3]:
# Ensure that we get deterministic results.
SEED=1
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [4]:
# Setup TensorBoard
writer = SummaryWriter(log_dir="/data/logs")

In [5]:
# Check that we have a CUDA enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
os.environ["MASTER_URL"] = "tcp://34.209.178.169:1234"
# Setup torch DDP
WORLD_SIZE = 2 # Number of nodes in cluster.
BACKEND = "nccl" # DDP backend.
URL = os.environ["MASTER_URL"] # Address of master node.
RANK = 1 # Rank of this node.
# Intialize distributed process group.
dist.init_process_group(init_method=URL, backend=BACKEND, world_size=WORLD_SIZE, rank=RANK)

In [7]:
MEAN_RGB = [0.47889522, 0.47227842, 0.43047404]
STD_RGB = [0.229, 0.224, 0.225]
IMG_SIZE = 128

In [8]:
# Load train and val dataset. 
TRAINDIR = "/data/train" # ImageNet train.
VALDIR = "/data/val" # ImageNet val.
BATCH_SIZE = 64
TRAIN_WORKERS = 6
VAL_WORKERS = 0
SHUFFLE = False

# Scale to ImageNet mean and STD since we will be using a model pretrained on ImageNet. 
transform_train = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(MEAN_RGB, STD_RGB),
])

transform_val = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(MEAN_RGB, STD_RGB),
])

# Load training data.
train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=BATCH_SIZE, num_workers=TRAIN_WORKERS, shuffle=SHUFFLE, pin_memory=True, 
        sampler=torch.utils.data.distributed.DistributedSampler(train_dataset)) # Sample data across nodes.

# Load validation data.
val_dataset = datasets.ImageFolder(
    VALDIR, transform=transform_val)

val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=BATCH_SIZE, num_workers=VAL_WORKERS, shuffle=SHUFFLE, pin_memory=True, 
        sampler=torch.utils.data.distributed.DistributedSampler(val_dataset)) # Sample data across nodes.

In [9]:
NUM_CLASSES = 1000
ARCH = 'resnet18'
LR = 1e-3
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
# Load model from PyTorch.
model = models.__dict__[ARCH]()
inf = model.fc.in_features
# Set fully connected layer to train with 1000 classes.
model.fc = nn.Linear(inf, NUM_CLASSES)
model.to(device)
# Setup optimizer and loss function. 
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)
criterion = nn.CrossEntropyLoss().to(device)

In [10]:
# Set CUDA device.
GPU_DEVICE_ID = 0
torch.cuda.set_device(GPU_DEVICE_ID)

In [11]:
# Wrap model in DDP. 
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[GPU_DEVICE_ID])

In [12]:
PRINT_FREQ = BATCH_SIZE - 1
global_step = 0

In [13]:
# Define train step.
def train(train_loader, model, criterion, optimizer, epoch):
    global global_step
    # Keep progress of metrics.
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # Switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # Measure data loading time.
        data_time.update(time.time() - end)
        
        # Move data to device
        images = images.to(device, non_blocking=True)
        target = target.to(device, non_blocking=True)
            
        # Compute output.
        output = model(images)
        loss = criterion(output, target)

        # Measure accuracy and record loss.
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))
        
        # Write tensorboard logs.
        writer.add_scalar("Loss/train", loss, global_step=global_step)
        writer.add_scalar("Acc1/train", top1.avg, global_step=global_step)
        writer.add_scalar("Acc5/train", top5.avg, global_step=global_step)
        global_step += 1

        # Compute gradient.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Measure elapsed time.
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [14]:
# Define validation step.
def validate(val_loader, model, criterion):
    global global_step
    # Keep progress of metrics.
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # Switch to evaluate mode.
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            
            # Move data to GPU if CUDA device is available.
            images = images.to(device, non_blocking=True)
            target = target.to(device, non_blocking=True)

            # Compute output.
            output = model(images)
            loss = criterion(output, target)

            # Measure accuracy and record loss.
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))
            

            # Measure elapsed time.
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
    
     # Write tensorboard logs.
    writer.add_scalar("Loss/val", loss, global_step=global_step)
    writer.add_scalar("Acc1/val", top1.avg, global_step=global_step)
    writer.add_scalar("Acc5/val", top5.avg, global_step=global_step)
    
    return top1.avg

In [15]:
for epoch in range(1):
    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    validate(val_loader, model, criterion)
    

Epoch: [0][    0/10010]	Time  1.274 ( 1.274)	Data  0.647 ( 0.647)	Loss 6.8940e+00 (6.8940e+00)	Acc@1   0.00 (  0.00)	Acc@5   1.56 (  1.56)
Epoch: [0][   63/10010]	Time  0.059 ( 0.117)	Data  0.001 ( 0.014)	Loss 7.0713e+00 (7.0176e+00)	Acc@1   0.00 (  0.05)	Acc@5   0.00 (  0.59)
Epoch: [0][  126/10010]	Time  0.290 ( 0.109)	Data  0.000 ( 0.008)	Loss 7.0336e+00 (6.9952e+00)	Acc@1   0.00 (  0.07)	Acc@5   0.00 (  0.52)
Epoch: [0][  189/10010]	Time  0.063 ( 0.104)	Data  0.000 ( 0.005)	Loss 6.9109e+00 (6.9782e+00)	Acc@1   0.00 (  0.08)	Acc@5   1.56 (  0.49)
Epoch: [0][  252/10010]	Time  0.066 ( 0.103)	Data  0.000 ( 0.004)	Loss 6.8843e+00 (6.9662e+00)	Acc@1   0.00 (  0.09)	Acc@5   0.00 (  0.52)
Epoch: [0][  315/10010]	Time  0.203 ( 0.102)	Data  0.000 ( 0.004)	Loss 6.9342e+00 (6.9554e+00)	Acc@1   0.00 (  0.11)	Acc@5   0.00 (  0.63)
Epoch: [0][  378/10010]	Time  0.062 ( 0.103)	Data  0.000 ( 0.003)	Loss 6.9124e+00 (6.9464e+00)	Acc@1   0.00 (  0.13)	Acc@5   1.56 (  0.70)
Epoch: [0][  441/10010]	Tim

Epoch: [0][ 3717/10010]	Time  0.136 ( 0.096)	Data  0.000 ( 0.005)	Loss 6.3085e+00 (6.5881e+00)	Acc@1   4.69 (  1.01)	Acc@5   4.69 (  3.51)
Epoch: [0][ 3780/10010]	Time  0.066 ( 0.096)	Data  0.000 ( 0.005)	Loss 6.2771e+00 (6.5811e+00)	Acc@1   1.56 (  1.02)	Acc@5   7.81 (  3.58)
Epoch: [0][ 3843/10010]	Time  0.224 ( 0.096)	Data  0.000 ( 0.005)	Loss 6.4742e+00 (6.5745e+00)	Acc@1   0.00 (  1.04)	Acc@5   4.69 (  3.64)
Epoch: [0][ 3906/10010]	Time  0.063 ( 0.096)	Data  0.000 ( 0.005)	Loss 6.2206e+00 (6.5679e+00)	Acc@1   3.12 (  1.05)	Acc@5   6.25 (  3.69)
Epoch: [0][ 3969/10010]	Time  0.156 ( 0.096)	Data  0.000 ( 0.005)	Loss 6.0316e+00 (6.5614e+00)	Acc@1   0.00 (  1.07)	Acc@5   9.38 (  3.75)
Epoch: [0][ 4032/10010]	Time  0.071 ( 0.096)	Data  0.000 ( 0.005)	Loss 6.1816e+00 (6.5549e+00)	Acc@1   0.00 (  1.09)	Acc@5   4.69 (  3.80)
Epoch: [0][ 4095/10010]	Time  0.070 ( 0.096)	Data  0.001 ( 0.005)	Loss 6.0534e+00 (6.5485e+00)	Acc@1   1.56 (  1.10)	Acc@5  10.94 (  3.86)
Epoch: [0][ 4158/10010]	Tim

Epoch: [0][ 7434/10010]	Time  0.061 ( 0.095)	Data  0.000 ( 0.005)	Loss 5.7652e+00 (6.2423e+00)	Acc@1   1.56 (  2.04)	Acc@5   6.25 (  6.77)
Epoch: [0][ 7497/10010]	Time  0.171 ( 0.095)	Data  0.000 ( 0.005)	Loss 5.5222e+00 (6.2372e+00)	Acc@1   4.69 (  2.06)	Acc@5  12.50 (  6.83)
Epoch: [0][ 7560/10010]	Time  0.067 ( 0.095)	Data  0.002 ( 0.005)	Loss 5.5798e+00 (6.2322e+00)	Acc@1   4.69 (  2.08)	Acc@5  14.06 (  6.89)
Epoch: [0][ 7623/10010]	Time  0.245 ( 0.095)	Data  0.001 ( 0.005)	Loss 5.7484e+00 (6.2270e+00)	Acc@1   1.56 (  2.09)	Acc@5   9.38 (  6.94)
Epoch: [0][ 7686/10010]	Time  0.206 ( 0.095)	Data  0.000 ( 0.005)	Loss 5.7168e+00 (6.2220e+00)	Acc@1   3.12 (  2.12)	Acc@5  10.94 (  6.99)
Epoch: [0][ 7749/10010]	Time  0.062 ( 0.095)	Data  0.000 ( 0.005)	Loss 5.4812e+00 (6.2171e+00)	Acc@1   6.25 (  2.13)	Acc@5  20.31 (  7.04)
Epoch: [0][ 7812/10010]	Time  0.237 ( 0.095)	Data  0.000 ( 0.005)	Loss 5.8376e+00 (6.2124e+00)	Acc@1   7.81 (  2.15)	Acc@5  15.62 (  7.09)
Epoch: [0][ 7875/10010]	Tim